In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.metrics import classification_report

In [72]:
df = pd.read_csv("valve_features.csv")
df.head()

,file_name,y_min,y_max,y_mean,y_std,zcr_mean,zcr_min,zcr_max,spec_cent_min,spec_cent_mean,spec_roll_min,spec_roll_mean,rms_mean,flat_mean,flat_max,amp_mean,amp_min,melspec_mean,melspec_std,class
0,0_dB_valve/valve/id_00/normal/00000773.wav,-0.078432,0.083072,2.846902e-06,0.005092,0.027839,0.007812,0.198730,534.462569,842.763456,861.328125,1565.728661,0.004988,0.000133,0.002137,0.012551,0.005413,0.005789,0.034216,0
1,0_dB_valve/valve/id_00/normal/00000632.wav,-0.074640,0.091593,-7.319340e-07,0.004906,0.077212,0.028809,0.214355,1289.842708,1608.087992,2680.883789,3643.133191,0.004762,0.000488,0.005983,0.014089,0.006951,0.005223,0.029242,0
2,0_dB_valve/valve/id_00/normal/00000238.wav,-0.067114,0.083659,2.985572e-07,0.005200,0.058272,0.031738,0.200684,893.401721,1333.155533,1819.555664,2813.130631,0.005101,0.000402,0.004332,0.015995,0.008361,0.005700,0.019618,0
3,0_dB_valve/valve/id_00/normal/00000148.wav,-0.074049,0.082879,-2.335126e-06,0.005244,0.063717,0.033691,0.197754,926.309215,1352.008946,1894.921875,2786.076734,0.005138,0.000408,0.005724,0.016069,0.005512,0.005686,0.017246,0
4,0_dB_valve/valve/id_00/normal/00000388.wav,-0.064568,0.075072,-7.828104e-08,0.005326,0.069678,0.029297,0.187500,1269.865358,1674.007315,2390.185547,3908.825938,0.005219,0.000483,0.005077,0.015526,0.007869,0.006105,0.029943,0


In [73]:
# Separate features and target
X = df.iloc[:,1:-1]
y = df.iloc[:,-1:]

In [74]:
# Split the data for train and test

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.3,random_state=1)
y_train.shape


(2919, 1)

In [75]:
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing dataabs
X_test_norm = norm.transform(X_test)

In [76]:
# data standardization with  sklearn
from sklearn.preprocessing import StandardScaler

# copy of datasets
X_train_stand = X_train.copy()
X_test_stand = X_test.copy()

# fit on training data column
scale = StandardScaler().fit(X_train_stand)
    
# transform the training data column
X_train_stand = scale.transform(X_train_stand)
    
# transform the testing data column
X_test_stand = scale.transform(X_test_stand)

In [77]:
# Use a KNN model
from sklearn.neighbors import KNeighborsClassifier

KNN_model=KNeighborsClassifier(n_neighbors=5)
KNN_model.fit(X_train_stand,y_train)
y_pred=KNN_model.predict(X_test_stand)

# Evaluate your model
from sklearn import metrics

print('Test accuracy for KNN is:',metrics.accuracy_score(y_test,y_pred)*100)
print('\nConfusion matrix:\n',metrics.confusion_matrix(y_test,y_pred))
print('\n Classification Report: \n',metrics.classification_report(y_test,y_pred))


/home/arfa/anaconda3/envs/sound/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Test accuracy for KNN is: 92.56594724220624

Confusion matrix:
 [[1095    6]
 [  87   63]]

 Classification Report: 
               precision    recall  f1-score   support

           0       0.93      0.99      0.96      1101
           1       0.91      0.42      0.58       150

    accuracy                           0.93      1251
   macro avg       0.92      0.71      0.77      1251
weighted avg       0.92      0.93      0.91      1251



Create a model for it to run

In [25]:
import pickle 

In [33]:
filename = 'valve_model.sav'
pickle.dump(KNN_model, open(filename, 'wb'))

In [44]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(X_test_stand)